In [ ]:
%%capture
!pip install langchain  openai duckduckgo-search youtube_search wikipedia langchain_experimental

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# Plan-and-Solve (PS) Prompting

Plan and Solve prompting was introduced in a May 2023  paper titled ["Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought Reasoning by Large Language Models](https://arxiv.org/abs/2305.04091), by Lei Wang et al.

### 🚧 **Why PS Prompting Matters:**
- Zero-shot-CoT is great but falls short sometimes with calculation errors, missing steps, and semantic misunderstandings.

- PS Prompting steps in to fix these issues.

### 🔍 **Breaking Down PS Prompting:**

<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2305.04091/assets/figures/ACL2023-math-cot-2.jpg" alt="Image Description" style="width:100%">
  <figcaption> Example inputs and outputs of GPT-3 with (a) Plan-and-Solve (PS) Prompting and (b) Plan-and-Solve prompting with more detailed instructions (PS+ prompting). PS+ prompting greatly improves the quality of the generated reasoning process..</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2305.04091">Image Source</a>
</figure>

1. 🗺️ **Planning**: The model first breaks down the task into smaller subtasks.

2. 🧩 **Solving**: Then, it works through these subtasks according to the plan.
   - Example: Instead of a vague "Let's think step by step," it uses a more structured approach like, "First, understand the problem and plan. Then, solve it step by step."

🚀 **PS+ Prompting - The Enhanced Version:**

- PS+ Prompting is a step up, adding detailed instructions.

- Focuses on avoiding calculation errors and refining reasoning steps.

- Guides the model to "identify relevant variables and numbers" and "compute intermediate results" for better accuracy and detail.

## Comparison with Other Methods

<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2305.04091/assets/figures/ACL2023-math-cot.jpg" alt="Image Description" style="width:100%">
  <figcaption>Example inputs and outputs of GPT-3 with (a) Zero-shot-CoT prompting, (b) Plan-and-Solve (PS) prompting, and (c) answer extraction prompting. While Zero-shot-CoT encourages LLMs to generate multi-step reasoning with “Let’s think step by step”, it may still generate wrong reasoning steps when the problem is complex. Unlike Zero-shot-CoT, PS prompting first asks LLMs to devise a plan to solve the problem by generating a step-by-step plan and carrying out the plan to find the answer.</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2305.04091">Image Source</a>
</figure>

 - Zero-shot-CoT: 💭 Thinks step by step, but can trip on complex problems.

- Few-shot-CoT: 📚 Uses example-based guidance for reasoning.

## ## Breaking Down Intuition & Results of PS Prompting

### 💡 **Why PS Prompting is Promising**

- 🏗️ **Structured Guidance:** Ensures step-by-step thinking and planning.

- 🎯 **Sharper Accuracy:** More coherent and precise reasoning chains.

### 📊 **Results Speak for Themselves:**

- 🥇 **Beats Zero-shot-CoT:** Outshines in various datasets.

- 🚀 **Rivals Other Methods:** Equally good, sometimes better than Zero-shot-Program-of-Thought Prompting.

- 💪 **Holds Up Against Few-Shot CoT:** Matches 8-shot CoT in arithmetic tasks.

### 🌟 **Final Takeaway:**

- 🧠 **Plan-and-Solve Prompting:** Offers a structured approach to eliciting detailed and accurate reasoning chains from large language models, making it a promising technique for multi-step reasoning tasks.


# Plan-and-Execute Agents in LangChain 🤖

### 🌟 **Inspiration & Approach:**

- **BabyAGI & "Plan-and-Solve" Roots:** Inspired by cutting-edge concepts and research.

- **Planning, Then Executing:** First, strategize; then, take action.

### 🛠️ **Key Components Explained:**

1. **The Planner:**

   - 🧠 Language Model at Work: Crafts a step-by-step strategy.

   - 🤔 Deals with Ambiguities: Navigates uncertainties in planning.

2. **The Executor:**
   - 🏃‍♂️ Action Agent in Play: Transforms plans into tangible actions.

   - 🔍 Chooses the Right Tools: Decides how best to achieve goals.


# Understanding Plan-and-Execute Agents 🤖

### 📋 **How They Work:**

1. **Planning Stage:**

   - 🧐 **Done by a Language Model (LLM):** Crafts a step-by-step strategy.

   - 🤔 **Dealing with Ambiguities:** Handles uncertainties and edge cases.

2. **Execution Stage:**

   - 🚀 **Taken Over by an Action Agent:** Transforms plans into actions.

   - 🛠️ **Determining Tools & Methods:** Chooses how best to reach goals.

### 🎯 **Why They're Special:**

- 🌍 **Complex Objectives:** Better at handling intricate tasks.

- 💪 **Increased Reliability:** Designed for dependable performance.

- 📈 **Scalability:** Can manage growing complexity and demands.

### ✅ **Benefits Over Traditional Agents:**

- 🧩 **Great for Complex Planning:** Manages multi-step tasks and dependencies.

- 👌 **Ideal for Multi-Tasking:** Handles tasks with several intertwined steps.

### 🤔 **When to Use Them:**

- **For Complex Goals:** If your task needs detailed planning and coordination.

- **Simple Objectives:** For simpler tasks, other agents might be more efficient.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents.tools import Tool
from langchain import LLMMathChain

# ⚒️ Define the Tools


In [ ]:
llm = OpenAI(temperature=0)

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

search = DuckDuckGoSearchRun()

search_tool = Tool(
    name="Search",
    func=search.run,
    description="Useful for answering questions about current events"
    )

calcuator_tool = Tool(
    name="Calculator",
    func=llm_math_chain.run,
    description="Useful for when you need to do math"
)

agent_tools = [search_tool, calcuator_tool]

# 🕵🏽 Create the Planner and Executor Agents

# 👩‍💼 Implement the planner


In [ ]:
model = ChatOpenAI(temperature=0)

planner = load_chat_planner(model)

In [ ]:
planner.llm_chain.prompt.messages[0].content

# 👨🏽‍💼 Instantiate the AgentExecutor


In [ ]:
executor = load_agent_executor(model, agent_tools, verbose=True)

agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [ ]:
print(executor.chain.agent.llm_chain.prompt.messages[0].prompt.template)

In [ ]:
agent.run("I need to know what the difference in current temperature is between Winnipeg, MB and Sacramento,CA")